## Original Dataset

### Generate 4 Band PNG (red, green, blue, nir) tiles for ACT

In [ ]:
%matplotlib inline

import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from numpngw import write_png

for j in range(18):
    for i in range(26):
        ds2018 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2018.nc")
        ds2019 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2019.nc")

        ds = xr.concat([ds2018, ds2019], dim='time').sortby('time')
        
        for t in ds.time:
            print(f"/data/pca_act/tiles/{j:03d}_{i:03d}_{str(t.values)[-30:-19]}.png")
            stack = np.dstack((ds.nbart_red.sel(time=t).values,
                               ds.nbart_green.sel(time=t).values,
                               ds.nbart_blue.sel(time=t).values,
                               ds.nbart_nir_1.sel(time=t).values))


            stack[np.isnan(stack)] = 10001
            write_png(f"/data/pca_act/tiles/{j:03d}_{i:03d}_{str(t.values)[-30:-19]}.png", stack.astype(np.uint16))

## Clouds removed, interpolated and rolled median Dataset

### Generate 3 Band PNG (red, green, blue) tiles for ACT

In [7]:
%matplotlib inline

import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from numpngw import write_png
from scipy import signal

def buffer_nans(da, kn):
    k = np.zeros((kn,kn))
    k[kn//2,kn//2] = 1

    arr = da.values
    mask = np.ones(arr.shape).astype(np.float32)

    for i in range(arr.shape[0]):
        mask[i,:,:] = signal.convolve2d(arr[i,:,:], k, boundary='fill', mode='same')

    return da.where(~np.isnan(mask))

for j in range(10,18):
    for i in range(26):
        """
        ds2018 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2018.nc")
        ds2019 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2019.nc")

        ds = xr.concat([ds2018, ds2019], dim='time').sortby('time')
        """
        ds = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2018.nc")
        
        
        blue = ds.nbart_blue.astype(np.float32) / 1e4

        # 1.- Filter reflectances greater than 0.5 
        blue = blue.where(blue<.5)

        # 2.- Filter reflectances with difference to lower quartile larger than 0.05 => (0.07)
        blue = blue.where((blue - blue.quantile(0.25, dim='time'))<.07)

        # 3.- Grow a 5x5 buffer around NaN pixels 
        blue = buffer_nans(blue, 5)

        # 4.- Discard frames with more than 25% missing pixels
        blue = blue.isel(time=(blue.count(dim=('x','y'))/(400*400))>.25)
        
        # 5. Apply blue mask over entire dataset
        ds = ds.sel(time=blue.time).where(~np.isnan(blue))
        
        print(ds.time)
        
        for band_name in ["nbart_red","nbart_green","nbart_blue"]:
            band = ds[band_name].astype(np.float32)# / 1e4

            # 1.- Interpolate NaNs over time linearly
            band = band.interpolate_na(dim='time')

            # 2.- Interpolate NaNs at the start and end using nearest neighbor
            band = band.interpolate_na(dim='time', method='nearest', fill_value='extrapolate')

            # 3.- Apply median rolling filter along time (window=3)
            band = band.rolling(time=3, min_periods=1).median()
            
            ds["clean_" + band_name] = band
        
        for t in ds.time[:6]:
            print(f"/data/pca_act/tiles/clean_{j:03d}_{i:03d}_{str(t.values)[-30:-19]}.png")
            stack = np.dstack((ds.clean_nbart_red.sel(time=t).astype(np.uint16).values,
                               ds.clean_nbart_green.sel(time=t).astype(np.uint16).values,
                               ds.clean_nbart_blue.sel(time=t).astype(np.uint16).values))

            write_png(f"/data/pca_act/tiles/clean_{j:03d}_{i:03d}_{str(t.values)[-30:-19]}.png", stack.astype(np.uint16))



<xarray.DataArray 'time' (time: 24)>
array(['2018-01-20T00:06:21.000000000', '2018-01-30T00:02:41.000000000',
       '2018-02-09T00:02:41.000000000', '2018-02-19T00:02:41.000000000',
       '2018-03-01T00:02:41.000000000', '2018-03-11T00:02:41.000000000',
       '2018-03-21T00:02:41.000000000', '2018-03-31T00:02:41.000000000',
       '2018-05-10T00:02:41.000000000', '2018-05-20T00:02:41.000000000',
       '2018-06-19T00:02:41.000000000', '2018-06-29T00:02:41.000000000',
       '2018-07-09T00:02:41.000000000', '2018-07-19T00:02:41.000000000',
       '2018-08-28T00:02:41.000000000', '2018-09-07T00:02:41.000000000',
       '2018-09-17T00:02:41.000000000', '2018-09-27T00:02:41.000000000',
       '2018-10-07T00:02:41.000000000', '2018-10-27T00:02:41.000000000',
       '2018-11-26T00:02:31.000000000', '2018-12-06T00:02:31.000000000',
       '2018-12-16T00:02:31.000000000', '2018-12-26T00:02:31.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2018

ValueError: x and y arrays must have at least 2 entries

## Clouds removed, raw Dataset

### Generate 4 Band PNG (red, green, blue, alpha) tiles for ACT

In [ ]:
%matplotlib inline

import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from numpngw import write_png
from scipy import signal

def buffer_nans(da, kn):
    k = np.zeros((kn,kn))
    k[kn//2,kn//2] = 1

    arr = da.values
    mask = np.ones(arr.shape).astype(np.float32)

    for i in range(arr.shape[0]):
        mask[i,:,:] = signal.convolve2d(arr[i,:,:], k, boundary='fill', mode='same')

    return da.where(~np.isnan(mask))

for j in range(18):
    for i in range(26):
        """
        ds2018 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2018.nc")
        ds2019 = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2019.nc")

        ds = xr.concat([ds2018, ds2019], dim='time').sortby('time')
        """
        ds = xr.open_dataset(f"/data/pca_act/{26*j+i:03d}_2018.nc")
        
        
        blue = ds.nbart_blue.astype(np.float32) / 1e4

        # 1.- Filter reflectances greater than 0.5 
        blue = blue.where(blue<.5)

        # 2.- Filter reflectances with difference to lower quartile larger than 0.05 => (0.07)
        blue = blue.where((blue - blue.quantile(0.25, dim='time'))<.07)

        # 3.- Grow a 5x5 buffer around NaN pixels 
        blue = buffer_nans(blue, 5)

        # 4.- Discard frames with more than 25% missing pixels
        blue = blue.isel(time=(blue.count(dim=('x','y'))/(400*400))>.25)
        
        # 5. Apply blue mask over entire dataset
        ds = ds.sel(time=blue.time).where(~np.isnan(blue))
        
        for t in ds.time[:6]:
            mask = np.isnan(blue.sel(time=t).values)
            mask = mask.astype(np.uint16) - 1
            
            print(f"/data/pca_act/tiles/clean_raw_{j:03d}_{i:03d}_{str(t.values)[-30:-19]}.png")
            stack = np.dstack((ds.nbart_red.sel(time=t).astype(np.uint16).values,
                               ds.nbart_green.sel(time=t).astype(np.uint16).values,
                               ds.nbart_blue.sel(time=t).astype(np.uint16).values,
                               mask))

            write_png(f"/data/pca_act/tiles/clean_raw_{j:03d}_{i:03d}_{str(t.values)[-30:-19]}.png", stack.astype(np.uint16))
